# Классификация клиентов телеком-компании

**Описание проекта:** Есть данные о поведении клиентов, которые перешли  тарифы «Смарт» и «Ультра». Нужно построить модель  классификации, которая выберет для клиента подходящий тариф. Данные уже предобработаны. 

**Цель проекта:** провести исследование нескольких моделей классификации с различными гиперпараметрами и определить лучшую модель. Значение доли правильных ответов (`accuracy`) на тестовой выборке должно быть не меньше `0.75`.

**Используемые данные:** информация об использовании разных видов трафика (в месяц) клиентами каждого тарифа.
* `calls` — количество звонков, совершённых клиентом;
* `minutes` — количество минут, потраченных клиентом;
* `messages` — количество сообщений, отправленных клиентом;
* `mb_used` — количество МБайт интернет-трафика, использованных клиентом;
* `is_ultra` — какой тариф использует клиент(`1` —  тариф «Ультра», `0` —  тариф «Смарт»).

## Оглавление
1. [Изучение данных](#1)
2. [Разделение данных на выборки](#2)
3. [Исследование моделей](#3)
4. [Проверка модели на тестовой выборке](#4)
5. [Проверка модели на адекватность](#5)

<a id="1"></a>
## 1. Изучение данных
Импортируем необходимые модули и классы, открываем файл и проверяем, что данные считались успешно.

In [1]:
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
data = pd.read_csv("/datasets/users_behavior.csv")
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Посмотрим на общую информацию о данных.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропусков в данных нет, приводить данные к каким-то другим типам нет необходимости. Посмотрим на статистические характеристики данных.

In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


Каких-то аномальных значений в данных не видно. Проверим данные на наличие дубликатов.

In [5]:
print(f"Количество дубликатов: {data.duplicated().sum()}")

Количество дубликатов: 0


Данные не требуют предобработки, можно переходить к разделению на выборки.

<a id="2"></a>
## 2. Разделение данных на выборки
Разделим данные на обучающую и тестовую.

In [6]:
X, y = data.drop(columns="is_ultra"), data["is_ultra"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Данные разбиты на обучающую и тестовую выборки. Теперь можно перейти к исследованию моделей.

<a id="3"></a>
## 3. Исследование моделей
Для исследования выберем два класса моделей: **случайный лес** (`RandomForestClassifier`) и **логистическую регрессию** (`LogisticRegression`). 

Будем использовать поиск по решётке с перекрёстной проверкой (`GridSearchCV`). Качество получившихся моделей будем оценивать с помощью доли правильных ответов (`accuracy`). Предварительно применим `StandartScaler`, чтобы отмасштабировать данные.

In [7]:
rf_clf = RandomForestClassifier(class_weight="balanced",
                                random_state=42)

logreg_clf = LogisticRegression(class_weight="balanced",
                                random_state=42)

params_logreg_l1 = {
    "classifier__C": [1, 0.1, 0.01],
    "classifier__penalty": ["l1"],
    "classifier__solver": ["liblinear"],
    "classifier": [logreg_clf]
}

params_logreg_l2 = {
    "classifier__C": [1, 0.1, 0.01],
    "classifier__penalty": ["l2"],
    "classifier": [logreg_clf]
}

params_logreg_el = {
    "classifier__C": [1, 0.1, 0.01],
    "classifier__penalty": ["elasticnet"],
    "classifier__solver": ["saga"],
    "classifier__l1_ratio": [0.25, 0.5, 0.75],
    "classifier": [logreg_clf]
}

params_rf = {
    "classifier__n_estimators": [100, 200, 300, 400],
    "classifier__max_depth": [4, 5, 6, 7],
    "classifier__criterion": ["gini", "entropy"],
    "classifier": [rf_clf]
}

params = [params_rf, params_logreg_l1, params_logreg_l2, params_logreg_el]

pipeline = Pipeline([("scaler", StandardScaler()),
                     ("classifier", rf_clf)])

grid_search = GridSearchCV(pipeline, params, scoring="accuracy", n_jobs=-1).fit(X_train, y_train)

Обучение моделей завершилось, выведем наилучшую модель на экран:

In [8]:
print("Лучшая модель")
grid_search.best_params_

Лучшая модель


{'classifier': RandomForestClassifier(class_weight='balanced', criterion='entropy',
                        max_depth=6, n_estimators=200, random_state=42),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 6,
 'classifier__n_estimators': 200}

Для более наглядного представления результатов напишем функцию и произведём дополнительные действия.

In [9]:
def add_model_class(classifier):
    if "RandomForest" in str(type(classifier)):
        return "RandomForestClassifier"
    else:
        return "LogisticRegression"
    

results = pd.DataFrame(grid_search.cv_results_)
results["model_class"] = results["param_classifier"].apply(add_model_class)

results = pd.DataFrame(results.groupby("model_class")
                              .max("mean_test_score")["mean_test_score"]
                              .sort_values(ascending=False))

results.columns = ["Среднее значение accuracy (CV)"]
results.index.name = "Тип модели"

In [10]:
results.head()

,Среднее значение accuracy (CV)
Тип модели,
RandomForestClassifier,0.799297
LogisticRegression,0.639056


Таким образом, лучшей моделью оказался случайный лес. Значение `accuracy` для этой модели получилось равно `0.799297`. Проверим эту модель на тестовой выборке.

<a id="4"></a>
## 4. Проверка модели на тестовой выборке
Используем лучшую модель на тестовой выборке и посчитаем долю правильных ответов.

In [11]:
print(f"accuracy на тестовой выборке: {accuracy_score(y_test, grid_search.predict(X_test)):.3f}")

accuracy на тестовой выборке: 0.807


`accuracy` получилась `0.807`. 

<a id="5"></a>
## 5. Проверка модели на адекватность
Проверим полученную модель на адекватность. В данном случае имеет смысл сравнить, насколько использованная модель лучше константной. Для этого воспользуемся классом `DummyClassifier` и установим `constant` в качестве стратегии прогноза, "обучим" эту модель и посчитаем `accuracy` на тестовой выборке.

In [12]:
dummy_clf = DummyClassifier(strategy="constant", random_state=42, constant=0)
dummy_clf.fit(X_train, y_train)
print(f"accuracy на тестовой выборке (DummyClassifier): {accuracy_score(y_test, dummy_clf.predict(X_test)):.3f}")

accuracy на тестовой выборке (DummyClassifier): 0.708


Получили, что `accuracy` примерно равна 0.7. Качество случайного леса, который обучали в предыдущих пунктах, выше этого значения, но отличие не столь велико. Вероятно, имеет смысл попробовать и другие модели, чтобы улучшить качество предсказаний. 